In [1]:
import pandas as pd
import datetime as dt

# Dates format

today_date = dt.date.today()
year_now = str(today_date.year)
year_int = int(year_now)
month_1 = '{:02d}'.format(today_date.month - 1)
month_2 = '{:02d}'.format(today_date.month - 2)
month_int = int(month_1)
month_2_int = int(month_2)
year_month = year_now + month_1

In [2]:
#Import file 1 as dataframe

df1 = pd.read_excel (r'C:\Users\juh_n\Documents\Python Scripts\File1_' + year_month + '.xlsx')
pd.set_option('float_format', '{:f}'.format)
df1.head(5)

,Date,Name,Balance
0,2020-08-31,NORTH,13187882447.440001
1,2020-08-31,EAST,1789935842.620000
2,2020-08-31,SOUTH,6651912475.050000
3,2020-08-31,NORTHEAST,3254304091.790000
4,2020-08-31,WEST,2930611472.910000


In [4]:
#Import File 2 as dataframe

df2 = pd.read_excel (r'C:\Users\juh_n\Documents\Python Scripts\File2_'+ year_month +'.xlsx')
pd.set_option('float_format', '{:f}'.format)
df2.head(5)

,Date,Name,TAX_ID,Fundo,Position,Central
0,2020-08-31,Nome,2843443000170,FI LIQUIDEZ,34951792.810000,SOUTH
1,2020-09-01,Nome,2843443000170,FI LIQUIDEZ,25087376.370000,SOUTH
2,2020-09-02,Nome,2843443000170,FI LIQUIDEZ,23646449.060000,SOUTH
3,2020-09-03,Nome,2843443000170,FI LIQUIDEZ,23137694.250000,SOUTH
4,2020-09-04,Nome,2843443000170,FI LIQUIDEZ,21457594.590000,SOUTH


In [9]:
#Group File 1 by date and name

df1_group = df1.groupby(['Date', 'Name']).mean()
df1_group.reset_index()


# Delete unwanted columns from File 2

df2_edit = df2.drop(['Name','TAX_ID','Fundo'],axis=1)
df2_edit.tail()

,Date,Position,Central
69760,2022-08-25,337565842.450000,SOUTH
69761,2022-08-26,337741986.930000,SOUTH
69762,2022-08-29,337915693.310000,SOUTH
69763,2022-08-30,338088597.460000,SOUTH
69764,2022-08-31,342098696.870000,SOUTH


In [21]:
# Deletes unwanted rows (with "Err" data) from "Central" column

del_err = df2_edit['Central']!= "Err"
df2_noErr = df2_edit[del_err]

In [22]:
# Groups File 2 by "Date" and "Central" then sums the values from column "Position"

df2_sum = df2_noErr.groupby(['Date', 'Central'])["Position"].sum().reset_index()
df2_sum.tail(5)

,Date,Central,Position
1912,2022-08-30,WEST,5368053494.140000
1913,2022-08-31,EAST,3203465281.410000
1914,2022-08-31,NORTH,8657501646.650000
1915,2022-08-31,SOUTH,13432742868.200001
1916,2022-08-31,WEST,6147792627.870000


In [23]:
# Merges both files by Date and Central

merge = pd.merge(df1, df2_sum, how='left',left_on=['Date','Name'], right_on=['Date', 'Central'])
merge.head(5)

,Date,Name,Balance,Central,Position
0,2020-08-31,NORTH,13187882447.440001,NaN,NaN
1,2020-08-31,EAST,1789935842.620000,EAST,2563165444.340000
2,2020-08-31,SOUTH,6651912475.050000,SOUTH,11862027272.840000
3,2020-08-31,NORTHEAST,3254304091.790000,NaN,NaN
4,2020-08-31,WEST,2930611472.910000,WEST,4481387820.780000


In [24]:
# Adds a new column called "Sum", which sums both values from columns "Balance" and "Position"

merge['Sum']= merge['Balance'] + merge.fillna(0)['Position']
merge.head()

,Date,Name,Balance,Central,Position,Sum
0,2020-08-31,NORTH,13187882447.440001,NaN,NaN,13187882447.440001
1,2020-08-31,EAST,1789935842.620000,EAST,2563165444.340000,4353101286.960000
2,2020-08-31,SOUTH,6651912475.050000,SOUTH,11862027272.840000,18513939747.889999
3,2020-08-31,NORTHEAST,3254304091.790000,NaN,NaN,3254304091.790000
4,2020-08-31,WEST,2930611472.910000,WEST,4481387820.780000,7411999293.690000


In [25]:
# Removes the columns we used to create the "Sum" (Balance + Position) and "Central"

merge = merge.drop(['Balance','Position', 'Central'],axis=1)
merge.head()

,Date,Name,Sum
0,2020-08-31,NORTH,13187882447.440001
1,2020-08-31,EAST,4353101286.960000
2,2020-08-31,SOUTH,18513939747.889999
3,2020-08-31,NORTHEAST,3254304091.790000
4,2020-08-31,WEST,7411999293.690000


In [26]:
# Convert the rows from column "Name" into 5 new columns

df = pd.pivot_table(merge, index=['Date'], columns=['Name'])
df = df.reset_index()
pd.set_option('float_format', '{:f}'.format)
df.tail()

Date               Sum                                       \
Name                         EAST              NORTH         NORTHEAST   
507  2022-08-25 6359544151.690001 16917598495.780001 3520903495.430000   
508  2022-08-26 6283243323.750000 16621931801.180000 3466037680.770000   
509  2022-08-29 6356342380.480000 16770752964.009998 3467232203.040000   
510  2022-08-30 6502421026.200001 16931736840.549999 3477421030.550000   
511  2022-08-31 6550712145.430000 17227423848.110001 3577229503.740000   

                                            
Name              SOUTH               WEST  
507  25872164396.959999  9603014719.119999  
508  25539980667.119999  9564616658.650000  
509  25439561198.980000  9891046339.500000  
510  25520787410.239998 10913093250.619999  
511  25786418724.520000 10730053695.680000

In [30]:
# Creates a new df with the daily variation of each Central

d=[]
for i in range(len(df.index)-1):
    p = []
    for u in range(df.shape[1]-1):
        p.append(df.loc[i+1][u+1]/df.loc[i][u+1])
    d.append(p)

In [31]:
# First line of new dataframe, which will be creating a forecast for the preview values

f = [[d[0][0]*df.loc[len(df.index)-1][1],d[0][1]*df.loc[len(df.index)-1][2],d[0][2]*df.loc[len(df.index)-1][3],d[0][3]*df.loc[len(df.index)-1][4],d[0][4]*df.loc[len(df.index)-1][5]]]
print(f)

[[6648081203.637951, 17251893969.74438, 3600427292.826084, 25971544782.255287, 10505716440.471104]]


In [32]:
# Creates the forecast dataframe, multiplying the value from D-1 by the variation (from the same day of preview year | Y-1) of each Central

for m in range(len(d)-1):
    g = []
    for n in range(len(d[0])):
        g.append(d[m+1][n]*f[m][n])
    f.append(g)

In [33]:
# Creates a list with the future business days

future_date = []
for i in range(len(f)):
    future_date.append(df.loc[len(df.index)-1][0] + pd.tseries.offsets.BusinessDay(n = i + 1))

In [36]:
# Creates a list separating month and year from the future business days

month_bd =[]
year_bd =[] 

for i in range(len(future_date)):
    month_bd.append(future_date[i].month)

for j in range(len(future_date)):
    year_bd.append(future_date[j].year)

df_bd = pd.DataFrame(f)
df_bd_future = pd.DataFrame(future_date)
df_bd_month = pd.DataFrame(month_bd)
df_bd_year = pd.DataFrame(year_bd)
df_bd_concat = pd.concat([df_bd_future, df_bd_month, df_bd_year, df_bd], axis=1)
df_bd_concat.columns = ['Date', 'Month', 'Year', 'EAST', 'WEST', 'NORTHEAST', 'NORTH', 'SOUTH']

# Forecast dataframe

df_bd_concat.head()

,Date,Month,Year,EAST,WEST,NORTHEAST,NORTH,SOUTH
0,2022-09-01,9,2022,6648081203.637951,17251893969.744381,3600427292.826084,25971544782.255287,10505716440.471104
1,2022-09-02,9,2022,6670149105.779974,17404170808.662212,3625990999.942112,26077990202.894302,10482277968.075829
2,2022-09-05,9,2022,6666983480.707208,17546796015.505833,3618490637.636442,26174630980.650093,10574777277.596937
3,2022-09-06,9,2022,6693023972.713823,17738158610.828922,3597933433.529525,26155627980.367558,10644807362.722481
4,2022-09-07,9,2022,6688300698.155133,17793327487.452740,3587480570.526355,26176445979.043205,10577575894.970755


In [37]:
# Group dataframe by Year and Month

df_ym = df_bd_concat.groupby(['Year','Month']).mean().reset_index()

 

# Filter year and month to forecast dates of the year you are in and present and future months

df_ym.query("Year == @year_int", inplace=True)
df_ym.query("Month >= @month_int", inplace=True)
df_ym.head()

,Year,Month,EAST,WEST,NORTHEAST,NORTH,SOUTH
0,2022,9,6618293196.299763,17696975782.496689,3641506676.374355,26053478205.603580,10314169414.503132
1,2022,10,6602603569.779820,17847855738.069733,3688336061.289461,25265158748.769157,10163063825.636311
2,2022,11,6950719063.143546,17709365018.416481,3722914123.291227,25491481493.153858,10121732110.920509
3,2022,12,6769589361.898868,17589403667.743168,3712322504.880171,24860880173.293148,9479881319.632231


In [ ]:
#df -> past
#df_bd_concat -> future
#df_ym -> forecast

In [39]:
# Save as Excel spreadsheet, alocating each dataframe in a different sheet (in this case, in the same folder that this project is saved):

with pd.ExcelWriter('Forecast_final.xlsx') as writer:
    df.to_excel(writer, sheet_name='Past')
    df_bd_concat.to_excel(writer, sheet_name='Future')
    df_ym.to_excel(writer, sheet_name='Forecast')

print('Sucess!')

Sucess!
